In [1]:
import os

if "jbook" in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))
import warnings

warnings.filterwarnings("ignore")

In [2]:
# from importlib import reload  # Not needed in Python 2
# import logging
# reload(logging)
# logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

# Normalization Stage

This normalization process involves standardizing data types, ensuring consistent encoding, and removing new lines from text.

In [3]:
from discover.container import DiscoverContainer
from discover.infra.config.orchestration import OrchestrationConfigReader
from discover.orchestration.data_prep.stage import DataPrepStage

## Dependency Container

In [4]:
container = DiscoverContainer()
container.init_resources()
container.wire(
    modules=[
        "discover.orchestration.data_prep.stage",
        "discover.orchestration.data_prep.dqa",
    ],
)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
# Obtain the normalization stage configuration
reader = OrchestrationConfigReader()
config = reader.get_config("phases", namespace=False)
stage_config = config["dataprep"]["stages"][1]

# Build and run the stage.
stage = DataPrepStage.build(stage_config=stage_config, force=True)
asset_id = stage.run()

[10/18/2024 03:29:59 PM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [_remove_dataset_file_by_filepath] : Removed dataset file at workspace/dev/dataset/01_dataprep/appvocai_discover-01_dataprep-01_normalize-review-dataset.parquet from repository.
[10/18/2024 03:29:59 PM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [remove] : Removed dataset dataset-dataprep-normalize-review from the repository.


Reset operation aborted.


# ============================================================================ #
#                           Data Normalization Stage                           #
# ============================================================================ #

Starting Data Normalization Stage Fri, 18 Oct 2024 15:29:59
Starting RemoveNewlinesTask Fri, 18 Oct 2024 15:30:00
Completed RemoveNewlinesTask Fri, 18 Oct 2024 15:30:00. Runtime: 0.04 seconds
Starting VerifyEncodingTask Fri, 18 Oct 2024 15:30:00
Completed VerifyEncodingTask Fri, 18 Oct 2024 15:30:00. Runtime: 0.33 seconds
Starting CastDataTypeTask Fri, 18 Oct 2024 15:30:00
Completed CastDataTypeTask Fri, 18 Oct 2024 15:30:00. Runtime: 0.06 seconds


                            Data Normalization Stage                            
                             Stage Start | Fri, 18 Oct 2024 15:29:59
                          Stage Complete | Fri, 18 Oct 2024 15:30:00
                                 Runtime | 0.97 seconds


In [6]:
repo = container.repo.dataset_repo()
ds = repo.get(asset_id=asset_id)
ds.content.info(verbose=True, memory_usage=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 59021 entries, 558124 to 20770924
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           59021 non-null  string        
 1   app_id       59021 non-null  string        
 2   app_name     59021 non-null  string        
 3   category_id  59021 non-null  category      
 4   author       59021 non-null  string        
 5   rating       59021 non-null  int16         
 6   content      59021 non-null  string        
 7   vote_sum     59021 non-null  int64         
 8   vote_count   59021 non-null  int64         
 9   date         59021 non-null  datetime64[ms]
 10  category     59021 non-null  category      
dtypes: category(2), datetime64[ms](1), int16(1), int64(2), string(5)
memory usage: 4.3 MB


In [7]:
ds.content.head()

,id,app_id,app_name,category_id,author,rating,content,vote_sum,vote_count,date,category
558124,7889871751,580643740,hoopla Digital,6018,0a433aa553dfe6554826,5,Titles I didn’t think they would have and a ap...,0,0,2021-10-08 01:25:00,Book
414067,6995102689,1076402606,"Libby, by OverDrive",6018,dce2976b6ee0e0fa44a0,5,I have been getting ebooks from the library fo...,0,0,2021-02-15 00:01:00,Book
418064,8308068603,1076402606,"Libby, by OverDrive",6018,50091da0238254dbef31,5,With my busy life the e- library brought back ...,0,0,2022-02-01 19:29:00,Book
571213,7316684624,852497554,Golden Quran | المصحف الذهبي,6018,8d51996c99eee37e1f9b,5,شكرا جزيلا لمصمم البرنامج والذين ساهمو فيه وا...,0,0,2021-05-09 09:14:00,Book
149022,8287426523,903001147,Axis 360,6018,ccbc729a023bde903a1a,3,This app has a lot of content to offer but I h...,0,0,2022-01-27 06:53:00,Book


Datatypes have been successfully cast, utf-8 encoding verified, and newlines have been removed, yielding a dataset of 22,166,591 observations and no null values.